<a href="https://colab.research.google.com/github/wonswe/DataAnalysis/blob/main/2-1%20%EB%AC%B8%EC%A0%9C%ED%92%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import requests

api_key = "4c54626d6c6a776b31334f6e686265"
base_url = "http://openapi.seoul.go.kr:8088"
# /{api_key}/json/energyUseDataSummaryInfo/{pageNo}/{pageNo + numOfRows - 1}/{year}/{month}

filtered_data = []
beg_year = 2015
end_year = 2024
beg_month = 1
end_month = 12
numOfRows = 1000
api_call_success = True

for year in range(beg_year, end_year + 1):
    for month in range(beg_month, end_month + 1):
        pageNo = 1
        while True:
            url = f"{base_url}/{api_key}/json/energyUseDataSummaryInfo/{pageNo}/{pageNo + numOfRows - 1}/{year}/{month:02d}"

            response = requests.get(url)

            if response.status_code == 200:
                try:
                    data = response.json().get('energyUseDataSummaryInfo')

                    if data and 'row' in data:
                        rows = data['row']
                        # MM_TYPE == "개인"
                        for row in rows:
                            if row.get('MM_TYPE') == '개인':
                                filtered_row = {
                                    '년도': row.get('YEAR'),
                                    '월': row.get('MON'),
                                    '전기': row.get('EUS'),
                                    '가스': row.get('GUS'),
                                    '수도': row.get('WUS'),
                                    '지역난방': row.get('HUS')
                                }
                                filtered_data.append(filtered_row)

                        if len(rows) < numOfRows:
                            break
                        else:
                            pageNo += numOfRows

                except requests.exceptions.JSONDecodeError as e:
                    print(f"JSON 디코딩 오류 ({year}년 {month}월): {e}")
                    break
            else:
                print(f"API 호출 실패 ({year}년 {month}월): {response.status_code}")
                api_call_success =  False
                break

if api_call_success:
    print("api 호출 성공!")
print("\n2015년 1월부터 2024년 12월까지의 개인 유형의 현년 전기, 가스, 수도, 지역난방 에너지 사용랑 데이터:")
print(filtered_data)
print(f"\n데이터 개수: {len(filtered_data)}")

# JSON 형태의 데이터를 pandas DataFrame으로 변환, 데이터의 기본 정보 출력:
import pandas as pd
df = pd.DataFrame(filtered_data)

# DataFrame 기본 정보 출력
print('\n<기본 정보 출력>')
print(df.info())
# DataFrame 기본 통계 출력
print('\n<기본 통계 출력>')
print(df.describe())
# DataFrame 인덱스 출력
print("\n<인덱스 출력>")
print(df.index)
# DataFrame 컬럼 출력
print("\n<컬럼 출력>")
print(df.columns.tolist())
# DataFrame 행 출력
print("\n<행 출력>")
print(df.values.tolist())
print("\n<위 5개의 행 출력>")
print(df.head())
print("\n<Display DataFrame>")
display(df)

api 호출 성공!

2015년 1월부터 2024년 12월까지의 개인 유형의 현년 전기, 가스, 수도, 지역난방 에너지 사용랑 데이터:
[{'년도': '2015', '월': '01', '전기': '193784708', '가스': '59133720', '수도': '12819757.886', '지역난방': '22740838.937'}, {'년도': '2015', '월': '02', '전기': '189974230', '가스': '56487358', '수도': '12656888.218', '지역난방': '18793320.213'}, {'년도': '2015', '월': '03', '전기': '175215719', '가스': '49984873', '수도': '12655474.249', '지역난방': '13826348.508'}, {'년도': '2015', '월': '04', '전기': '183891769', '가스': '37306841', '수도': '12954773.246', '지역난방': '5836002.885'}, {'년도': '2015', '월': '05', '전기': '177289769', '가스': '22060892', '수도': '13086734.214', '지역난방': '1320895.801'}, {'년도': '2015', '월': '06', '전기': '186073153', '가스': '12677645', '수도': '14241651.402', '지역난방': '294312.634'}, {'년도': '2015', '월': '07', '전기': '165114568', '가스': '8893640', '수도': '12083879.038', '지역난방': '702863.645'}, {'년도': '2015', '월': '08', '전기': '242000479', '가스': '7619018', '수도': '15164553.742', '지역난방': '460288.965'}, {'년도': '2015', '월': '09', '전기': '207540868', '가스': '6

,년도,월,전기,가스,수도,지역난방
0,2015,01,193784708,59133720,12819757.886,22740838.937
1,2015,02,189974230,56487358,12656888.218,18793320.213
2,2015,03,175215719,49984873,12655474.249,13826348.508
3,2015,04,183891769,37306841,12954773.246,5836002.885
4,2015,05,177289769,22060892,13086734.214,1320895.801
...,...,...,...,...,...,...
115,2024,08,463971328,7072251,15873544.6,130082.263
116,2024,09,409872908,5729131,15143321.6,3420507.136
117,2024,10,282268119,8233421,15297669.9,2695491.396
118,2024,11,257927956,17418221,14678672.5,8836296.454
